# Introduction
This project provides an automated image processing pipeline designed to detect and center people in photos. Using YOLOv8 for detection and OpenCV for manipulation, the pipeline ensures consistent output by resizing, cropping, and centering individuals in images. Key features include:

*   Precise Detection: Identifies and localizes people in images.
*   Scaling and Cropping: Adjusts images to a fixed aspect ratio (3:4) and centers the subject.
*   Easy Configuration: Customizable parameters for bounding box height and crop dimensions.

This notebook enables quick setup and execution of the pipeline directly in Colab, making it ideal for processing images with minimal local setup.

# Preparation Step: Install Required Libraries


Before running the image processing pipeline, we need to ensure all necessary libraries are installed. The following lines perform the setup:

```
!pip install opencv-python opencv-python-headless
```

* **Installs OpenCV**: This library is used for reading, manipulating, and saving images. The opencv-python-headless variant is designed for environments without GUI support, such as Colab.

```
!pip install mediapipe`
```

* **Installs MediaPipe**: Although not directly used in this script, MediaPipe is a framework for building multimodal (e.g., face, hands, or pose) tracking solutions and might complement YOLO-based detection in future extensions.

```
!pip install ultralytics
from ultralytics import YOLO
```


* **Installs Ultralytics**: Provides access to the YOLOv8 object detection framework, which is essential for detecting persons in the images.
Runtime Environment Settings
Ensure your Colab runtime is configured as follows:

* **Runtime Type**: Python 3
* **Hardware Accelerator**: CPU

These settings are sufficient for the pipeline since the processing is optimized for environments without GPU support. If GPU-based enhancements (e.g., Real-ESRGAN) are integrated in the future, the hardware accelerator can be switched to GPU.

In [1]:
!pip install opencv-python opencv-python-headless
!pip install mediapipe
!pip install ultralytics
from ultralytics import YOLO

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.8/906.8 kB 16.1 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


# Configuration and Global Variables
This section sets up key parameters for the image processing pipeline:


```
desired_box_height = 2600
crop_width = 1920
crop_height = 2700
```

* **desired_box_height**: Ensures the detected person occupies 2600 pixels vertically in the final image. This leaves 50 pixels margin above and below the bounding box for a total height of 2700 pixels.
* **crop_width and crop_height**: Define the final output image dimensions, maintaining a 3:4 aspect ratio.


```
INPUT_FOLDER = 'input_images'
OUTPUT_FOLDER = 'output_images'
```

* INPUT_FOLDER: Specifies the folder containing images to process.
* OUTPUT_FOLDER: Specifies where the processed images will be saved.
The working_images subfolder stores intermediate results, such as bounding box annotations and upscaled images.

# Intermediate Steps and Notes
The processing pipeline generates intermediate outputs for debugging and visualization:


1. **Bounding Boxes**: Stored in the working_images folder, these images show how the system detects and annotates persons.
2. **Upscaled Images**: Use bicubic interpolation for resizing:

```
upscaled_img = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_CUBIC)
```

* **Note**: AI-based upscaling (e.g., Real-ESRGAN) was considered but omitted due to:
 * GPU requirements.
 * Longer processing times.
 * Image artifacts and loss of detail in textures like linen or silk.

 Future enhancements may replace this step with a more efficient AI-based method, ensuring better quality while maintaining acceptable performance.

In [3]:
#final code
import os
import cv2
from ultralytics import YOLO
import matplotlib.pyplot as plt

# Global variables
# Desired box height for scaling
desired_box_height = 2600
# Crop dimensions maintaining a 3:4 aspect ratio
crop_width = 1920
crop_height = 2700
INPUT_FOLDER = 'input_images'
OUTPUT_FOLDER = 'output_images'

# Utility functions
def visualize_image(image, comment):
    """Display an image with a comment."""
    print(comment)
    plt.figure(figsize=(10, 10))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB for visualization
    plt.axis('off')
    plt.show()

def save_image_with_bounding_box(image, box, output_path, comment=""):
    """Draw a bounding box on the image and save it."""
    x_min, y_min, box_width, box_height = box
    img_with_box = image.copy()
    cv2.rectangle(img_with_box, (x_min, y_min), (x_min + box_width, y_min + box_height), (0, 255, 0), 2)
    cv2.imwrite(output_path, img_with_box)
    print(f"{comment} {output_path}")

def detect_person_with_yolo(image):
    """Detect a person in the image using YOLO."""
    try:
        model = YOLO('yolov8n.pt')  # Use YOLOv8 nano model
        results = model(image)
        for result in results[0].boxes.data.tolist():
            x_min, y_min, x_max, y_max, confidence, class_id = result
            if int(class_id) == 0:  # Class "person" in YOLO
                box_width = int(x_max - x_min)
                box_height = int(y_max - y_min)
                return [int(x_min), int(y_min), box_width, box_height]
        return None  # No person found
    except Exception as e:
        print(f"Error finding person with YOLO: {e}")
        return None

def calculate_scaling_factor(box_height):
    """Calculate the scaling factor to achieve the desired bounding box height."""
    return desired_box_height / box_height

def crop_image_to_center(image, box):
    """Crop the image around the bounding box, centering it."""
    x_min, y_min, box_width, box_height = box
    center_x = x_min + box_width // 2
    center_y = y_min + box_height // 2
    crop_center_x = crop_width // 2
    crop_center_y = crop_height // 2
    offset_x = center_x - crop_center_x
    offset_y = center_y - crop_center_y

    crop_x_min = max(0, offset_x)
    crop_y_min = max(0, offset_y)
    crop_x_max = min(crop_x_min + crop_width, image.shape[1])
    crop_y_max = min(crop_y_min + crop_height, image.shape[0])
    crop_x_min = crop_x_max - crop_width
    crop_y_min = crop_y_max - crop_height

    return image[crop_y_min:crop_y_max, crop_x_min:crop_x_max]

# Main processing functions
def process_image(input_path, output_folder, working_folder):
    """Process a single image: detect, upscale, crop, and save intermediate results."""
    if not os.path.isfile(input_path):
        print(f"Error: File {input_path} does not exist.")
        return

    os.makedirs(output_folder, exist_ok=True)
    os.makedirs(working_folder, exist_ok=True)

    base_name = os.path.splitext(os.path.basename(input_path))[0]
    img = cv2.imread(input_path)
    if img is None:
        print(f"Error: Unable to read the file {input_path}.")
        return

    person_box = detect_person_with_yolo(img)
    if not person_box:
        print("Error: No person detected.")
        return

    save_image_with_bounding_box(img, person_box, os.path.join(working_folder, f"{base_name}_step1_bounding_box.jpg"), "Saved image with bounding box:")

    scaling_factor = calculate_scaling_factor(person_box[3])
    new_width = int(img.shape[1] * scaling_factor)
    new_height = int(img.shape[0] * scaling_factor)
    upscaled_img = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_CUBIC)

    person_box = detect_person_with_yolo(upscaled_img)
    save_image_with_bounding_box(upscaled_img, person_box, os.path.join(working_folder, f"{base_name}_step2_upscaled_bounding_box.jpg"), "Saved upscaled image with bounding box:")

    cropped_img = crop_image_to_center(upscaled_img, person_box)
    cv2.imwrite(os.path.join(working_folder, f"{base_name}_step3_cropped_image.jpg"), cropped_img)
    print(f"Saved cropped image: {os.path.join(working_folder, f'{base_name}_step3_cropped_image.jpg')}")

    cv2.imwrite(os.path.join(output_folder, f"{base_name}.jpg"), cropped_img)
    print(f"Final processed image saved as: {os.path.join(output_folder, f'{base_name}.jpg')}" )

def process_images_in_folder(input_folder, output_folder):
    """Process all images in the input folder and organize results."""
    if not os.path.exists(input_folder):
        print(f"Error: Input folder {input_folder} does not exist.")
        return

    working_folder = os.path.join(output_folder, "working_images")
    os.makedirs(working_folder, exist_ok=True)

    for file_name in os.listdir(input_folder):
        input_path = os.path.join(input_folder, file_name)
        if os.path.isfile(input_path):
            process_image(input_path, output_folder, working_folder)

# Main function
process_images_in_folder(INPUT_FOLDER, OUTPUT_FOLDER)


0: 384x640 1 person, 268.5ms
Speed: 29.1ms preprocess, 268.5ms inference, 7.4ms postprocess per image at shape (1, 3, 384, 640)
Saved image with bounding box: output_images/working_images/depositphotos_509201804-stock-photo-business-man-in-shirt-with_step1_bounding_box.jpg

0: 384x640 1 person, 202.0ms
Speed: 7.8ms preprocess, 202.0ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)
Saved upscaled image with bounding box: output_images/working_images/depositphotos_509201804-stock-photo-business-man-in-shirt-with_step2_upscaled_bounding_box.jpg
Saved cropped image: output_images/working_images/depositphotos_509201804-stock-photo-business-man-in-shirt-with_step3_cropped_image.jpg
Final processed image saved as: output_images/depositphotos_509201804-stock-photo-business-man-in-shirt-with.jpg
